In [2]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import processing_netcdf as pcdf
import geopandas as gpd
from geopandas import GeoDataFrame
import shapely.geometry 
import numpy as np
from shapely import geometry as gmty
from geofeather import to_geofeather, from_geofeather
import glob
import os
import pyarrow
from xclim import ensembles as ens
from xclim import subset
import matplotlib.pyplot as plt

In [ ]:
folder = "/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/"

# Select variable

In [ ]:
#Variable 
variable85="rcp85_tg_mean_annual"
variable45="rcp45_tg_mean_annual"
#variable="rcp[48]5_tg_mean_annual"

files85 = glob.glob(folder+"*"+variable85+".nc")
files45 = glob.glob(folder+"*"+variable45+".nc")
#ex: ACCESS1-3_rcp45_tn_mean_annual.nc
files85
files45

# FOR RCP 45

## Subsetting arrays by periods of time and percentiles with Xclim

In [ ]:
dsEns45= ens.create_ensemble(files45)
dsEns45

In [ ]:
# Mapping data - 30 year means
tmp145 = dsEns45.sel(time=(dsEns45.time.dt.year>=1981))
window = 30
time145 = tmp145.time[0::window]

ds30yavg45 = tmp145.coarsen(time=window).mean()
ds30yavg45['time'] = time145
perc30yavg45 = ens.ensemble_percentiles(ds30yavg45)
perc30yavg45 # entire grid

# Mapping data

## Transform into DataFrame

In [ ]:
%time df45 = perc30yavg45.drop('realization').to_dataframe().dropna()

df45

## Transform in Celsius and round

In [ ]:
df45C = df45.copy()
df45C["tg_mean_p10"] = round(df45C["tg_mean_p10"] -273.15,2)
df45C["tg_mean_p50"] = round(df45C["tg_mean_p50"] -273.15,2)
df45C["tg_mean_p90"] = round(df45C["tg_mean_p90"] -273.15,2)

df45C

## Eliminate 2011, Pivot Table and Create new columns 

In [ ]:
Region1i45 = df45C.reset_index()
print (Region1i45)
Region1i245 = Region1i45[Region1i45.time.dt.year!= 2011]
print ("2011 eliminated")
df45f = Region1i245.pivot_table(index=["lat","lon"], columns="time")
print ("DF pivoted")
df45f.columns = [year + "rcp45_p"+p
                     for p in ["10", "50", "90"]
                     for year in ["hist_","t2050_", "t2080_"]]
print("Columns renamed")
df45f

# FOR RCP 85

## Subsetting arrays by periods of time and percentiles with Xclim

In [ ]:
dsEns85= ens.create_ensemble(files85)
dsEns85

In [ ]:
# Mapping data - 30 year means
tmp185 = dsEns85.sel(time=(dsEns85.time.dt.year>=1981))
window = 30
time185 = tmp185.time[0::window]

ds30yavg85 = tmp185.coarsen(time=window).mean()
ds30yavg85['time'] = time185
perc30yavg85 = ens.ensemble_percentiles(ds30yavg85)
perc30yavg85 # entire grid

## Transform into DataFrame

In [ ]:
%time df85 = perc30yavg85.drop('realization').to_dataframe().dropna()
df85

## Transform in Celsius and round

In [ ]:
df85C = df85.copy()
df85C["tg_mean_p10"] = round(df85C["tg_mean_p10"] -273.15,2)
df85C["tg_mean_p50"] = round(df85C["tg_mean_p50"] -273.15,2)
df85C["tg_mean_p90"] = round(df85C["tg_mean_p90"] -273.15,2)

df85C

## Eliminate 2011, Pivot Table and Create new columns 

In [ ]:
Region1i85 = df85C.reset_index()
print (Region1i85)
Region1i285 = Region1i85[Region1i85.time.dt.year!= 2011]
print ("2011 eliminated")
df85f = Region1i285.pivot_table(index=["lat","lon"], columns="time")
print ("DF pivoted")
df85f.columns = [year + "rcp85_p"+p
                     for p in ["10", "50", "90"]
                     for year in ["hist_","t2050_", "t2080_"]]
print("Columns renamed")
df85f

# Merge df45 and 85

In [ ]:
dftg = pd.merge(df45f, df85f, on=["lat","lon"])
dftg.head()

# Extract hist

In [ ]:
dftg1 = dftg.drop(columns=["hist_rcp45_p10","hist_rcp45_p50", "hist_rcp45_p90", "hist_rcp85_p10", "hist_rcp85_p50", "hist_rcp85_p90"])
dftgh = dftg[["hist_rcp45_p50", "hist_rcp85_p50"]]

In [ ]:
dftg1

# Merge both hist_50 and get the mean

In [ ]:
dftgh["hist_p50"] = round((dftgh["hist_rcp45_p50"]+ dftgh["hist_rcp85_p50"])/2, 2)
dftgh2 = dftgh.reset_index()

In [ ]:
dftgh3 = dftgh2[["lat", "lon", "hist_p50"]]
dftgh3

# Merge means with the 45 and 85 DF

In [ ]:
dftg_all = pd.merge(dftgh3, dftg1, on=["lat","lon"])

In [ ]:
dftg_all

In [ ]:
dftg_all.to_feather("tg_mean_annual.feather")

In [3]:
dftg_all = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/tg_mean_annual.feather")

In [4]:
dftg_all 

,lat,lon,hist_p50,t2050_rcp45_p10,t2080_rcp45_p10,t2050_rcp45_p50,t2080_rcp45_p50,t2050_rcp45_p90,t2080_rcp45_p90,t2050_rcp85_p10,t2080_rcp85_p10,t2050_rcp85_p50,t2080_rcp85_p50,t2050_rcp85_p90,t2080_rcp85_p90
0,40.041039,-89.045212,11.64,12.67,13.07,13.78,14.34,14.27,14.75,13.51,14.80,14.69,16.66,14.88,17.00
1,40.041039,-88.961884,11.61,12.66,13.05,13.76,14.32,14.25,14.73,13.49,14.78,14.67,16.64,14.86,16.98
2,40.041039,-88.878548,11.60,12.65,13.04,13.75,14.32,14.24,14.72,13.48,14.77,14.66,16.64,14.85,16.97
3,40.041039,-88.795219,11.58,12.62,13.02,13.73,14.30,14.22,14.70,13.46,14.75,14.64,16.62,14.83,16.95
4,40.041039,-88.711891,11.60,12.64,13.04,13.75,14.32,14.24,14.72,13.48,14.77,14.65,16.64,14.85,16.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,-9.63,-8.06,-8.27,-7.34,-6.49,-6.57,-5.77,-7.19,-6.27,-5.92,-3.81,-5.20,-2.56
76202,66.623306,-61.546310,-9.92,-8.37,-8.57,-7.63,-6.79,-6.88,-6.08,-7.49,-6.58,-6.22,-4.12,-5.57,-2.95
76203,66.623306,-61.462978,-9.71,-8.15,-8.36,-7.40,-6.57,-6.67,-5.88,-7.28,-6.37,-6.01,-3.92,-5.43,-2.82
76204,66.623306,-61.379650,-9.63,-8.08,-8.28,-7.31,-6.49,-6.66,-5.80,-7.20,-6.29,-5.93,-3.85,-5.42,-2.83


# Merge complete DF with Polygons for each spatial scale

## Dictionary - regioins: column name, short name

SyntaxError: invalid syntax (<ipython-input-5-9c8e998f978d>, line 1)

In [6]:
short_dict = {"DDE_STF_20K_REG_FOR_VUE_S": ("NM_REG_FOR", "RF"), 
              "DDE_STF_20K_UA_PER_VUE_S": ("PER_NO_UA", "UA"), 
              "DOM_BIO": ("NOM", "DB"), 
              "REG_ECO": ("NOM", "RE"), 
              "SDOM_BIO": ("NOM", "SDB"), 
              "Secteurs_Operations_Regionales": ("D_GENERAL", "SOR"), 
              "SREG_ECO": ("NOM", "SRE"), 
              "territoire_guide": ("TER_GUIDE", "TG") 
              }

for region, (name, short) in short_dict.items():
    print(region, name, short)

DDE_STF_20K_REG_FOR_VUE_S NM_REG_FOR RF
DDE_STF_20K_UA_PER_VUE_S PER_NO_UA UA
DOM_BIO NOM DB
REG_ECO NOM RE
SDOM_BIO NOM SDB
Secteurs_Operations_Regionales D_GENERAL SOR
SREG_ECO NOM SRE
territoire_guide TER_GUIDE TG


In [7]:
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

In [8]:
d = { 'BAS-SAINT-LAURENT':"Bas-Saint-Laurent", 
     'SAGUENAY -LAC-SAINT-JEAN': "Saguenay -Lac-Saint-Jean",
     'CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES':"Capitale-Nationale-Chaudiere-Appalaches",
     'MAURICIE-CENTRE-DU-QUÉBEC':'Mauricie-Centre-du-Quebec','OUTAOUAIS':'Outaouais', 
     'ABITIBI-TEMISCAMINGUE':'Abitibi-Temiscamingue', 'COTE-NORD':'Cote-Nord',
     'NORD-DU-QUEBEC':'Nord-Du-Quebec', 'GASPESIE-ILES-DE-LA-MADELEINE':'Gaspesie-Iles-De-La-Madeleine', 
     'LANAUDIERE':'Lanaudiere','LAURENTIDES':"Laurentides", 
     "ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL":"Estrie-Monteregie-Laval-Montreal", 
     "é": "e", "à": "a", "è": "e", "ô" : "o", "ç":"c", "É": "E", "È": "E", "Î": "i", "Ç":"C" } 

In [9]:
for region, (name, short) in short_dict.items():
    print(region)
    dfpolyshape = from_geofeather('/home/mlopez/EXEC/Grids-polygons-regions/Grid-'+region+'.feather')
    dftp = pd.merge(dftg_all, dfpolyshape, on=["lat","lon"])
    print ("Merged with polygons")
    listTG = []
    for tg in dftp[name].unique().tolist():
        df2 = dftp[dftp[name] == tg]
        print (tg)
        if tg != None:
            listTG.append(df2)
        #print (listTG)
    for df in listTG:
        geometry = df["geometry"]
        crs = {'init': "epsg:4326"}
        gdf = GeoDataFrame(df, crs=crs, geometry=geometry)
        print (gdf[name].iloc[0])
        #Substitute filename accents
        gdf.to_file(replace_all("/home/mlopez/EXEC/GeoJsonMFFP/"+gdf[name].iloc[0], d)+"_tg_mean_annual.json", driver="GeoJSON")
    geometry = dftp["geometry"]
    crs = {'init': "epsg:4326"}
    gdf = GeoDataFrame(dftp, crs=crs, geometry=geometry)
    gdf.to_file("/home/mlopez/EXEC/GeoJsonMFFP/"+short+"_tg_mean_annual.json", driver="GeoJSON")

  warnings.warn(



DDE_STF_20K_REG_FOR_VUE_S
Merged with polygons
ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL
OUTAOUAIS
LAURENTIDES
MAURICIE-CENTRE-DU-QUÉBEC
LANAUDIERE
CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES
ABITIBI-TEMISCAMINGUE
BAS-SAINT-LAURENT
GASPESIE-ILES-DE-LA-MADELEINE
SAGUENAY -LAC-SAINT-JEAN
COTE-NORD
NORD-DU-QUEBEC
ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL
OUTAOUAIS
LAURENTIDES
MAURICIE-CENTRE-DU-QUÉBEC
LANAUDIERE
CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES
ABITIBI-TEMISCAMINGUE
BAS-SAINT-LAURENT
GASPESIE-ILES-DE-LA-MADELEINE
SAGUENAY -LAC-SAINT-JEAN
COTE-NORD
NORD-DU-QUEBEC


  warnings.warn(



DDE_STF_20K_UA_PER_VUE_S
Merged with polygons
05151
07151
07251
06151
03571
06452
06271
07351
07152
08151
06471
04151
03451
08152
07352
03171
07451
04352
04251
01171
03153
08351
02371
03351
08251
04351
01272
08451
02571
11161
09751
11262
08462
08651
02651
08751
08551
02471
11263
08762
02751
02664
09351
08652
02666
08763
09471
08764
02665
08665
02663
08562
08664
08666
02662
08663
09551
02661
09352
05151
07151
07251
06151
03571
06452
06271
07351
07152
08151
06471
04151
03451
08152
07352
03171
07451
04352
04251
01171
03153
08351
02371
03351
08251
04351
01272
08451
02571
11161
09751
11262
08462
08651
02651
08751
08551
02471
11263
08762
02751
02664
09351
08652
02666
08763
09471
08764
02665
08665
02663
08562
08664
08666
02662
08663
09551
02661
09352


  warnings.warn(



...

Massif des monts Groulx


ERROR:fiona._env:Coteaux des lacs Matonipi et Jonquet_tg_mean_annual.json: No such file or directory


Coteaux des lacs Matonipi et Jonquet


  warnings.warn(



territoire_guide
Merged with polygons
1a
2c
3d
2b
2a
3c
3ab
4bc
4de
5ef
4f
4a
5jk
5bcd
5a
4gh
5hi
5g
6cdefg
6hi
6ab
6j
6mn
6kl
6opqr
1a
2c
3d
2b
2a
3c
3ab
4bc
4de
5ef
4f
4a
5jk
5bcd
5a
4gh
5hi
5g
6cdefg
6hi
6ab
6j
6mn
6kl
6opqr


## Select and open Region (Shapefile)

In [66]:
#region = "DDE_STF_20K_REG_FOR_VUE_S"
             

## Import dataframe with polygons

In [67]:
dfpolyshape = from_geofeather('/home/mlopez/EXEC/Grids-polygons-regions/Grid-'+region+'.feather')
#dfpolyshape = from_geofeather('/home/mlopez/EXEC/Grids-polygons-regions/Grid-.feather')

In [68]:
dfpolyshape.head()

,lat,lon,OBJECTID,NO_REG_FOR,NM_REG_FOR,NO_UG_RAF,SUP_KM2,DATE_PUBLI,Shape_Leng,Shape_Area,D_GENERAL,geometry
0,53.040520,-66.462784,8,09,COTE-NORD,"093, 094, 097",379435.8,2019-11-30,5.360494e+06,3.744732e+11,Nord-est,"MULTIPOLYGON (((-66.42112 53.03552, -66.42112 ..."
1,53.040520,-66.379448,8,09,COTE-NORD,"093, 094, 097",379435.8,2019-11-30,5.360494e+06,3.744732e+11,Nord-est,"POLYGON ((-66.42111 52.99885, -66.42111 53.035..."
2,52.957191,-66.629440,8,09,COTE-NORD,"093, 094, 097",379435.8,2019-11-30,5.360494e+06,3.744732e+11,Nord-est,"POLYGON ((-66.58778 52.94277, -66.58778 52.915..."
3,52.957191,-66.546112,8,09,COTE-NORD,"093, 094, 097",379435.8,2019-11-30,5.360494e+06,3.744732e+11,Nord-est,"POLYGON ((-66.58778 52.91553, -66.58778 52.942..."
4,52.957191,-66.462784,8,09,COTE-NORD,"093, 094, 097",379435.8,2019-11-30,5.360494e+06,3.744732e+11,Nord-est,"POLYGON ((-66.50445 52.91553, -66.50445 52.948..."


In [49]:
dftp = pd.merge(dftg_all, dfpolyshape, on=["lat","lon"])
print ("Merged with polygons")
dftp.head()

Merged with polygons


,lat,lon,hist_p50,t2050_rcp45_p10,t2080_rcp45_p10,t2050_rcp45_p50,t2080_rcp45_p50,t2050_rcp45_p90,t2080_rcp45_p90,t2050_rcp85_p10,...,ZONE_VEG,SZONE_VEG,DOM_BIO,SDOM_BIO,REG_ECO,NOM,SUPERFICIE,Shape_Leng,Shape_Area,geometry
0,44.957512,-74.629120,7.26,8.70,8.88,9.69,10.34,10.34,10.98,9.71,...,Z1,Z11,1,1,1a,Plaine du bas Outaouais et de l'archipel de Mo...,1216980.4,8.484385,1.396666,"MULTIPOLYGON (((-74.58955 44.99918, -74.58746 ..."
1,44.957512,-74.545792,7.23,8.67,8.85,9.67,10.32,10.31,10.96,9.68,...,Z1,Z11,1,1,1a,Plaine du bas Outaouais et de l'archipel de Mo...,1216980.4,8.484385,1.396666,"POLYGON ((-74.58746 44.99916, -74.58746 44.999..."
2,44.957512,-74.462463,7.20,8.65,8.82,9.64,10.30,10.29,10.93,9.65,...,Z1,Z11,1,1,1a,Plaine du bas Outaouais et de l'archipel de Mo...,1216980.4,8.484385,1.396666,"POLYGON ((-74.50413 44.99792, -74.50413 44.999..."
3,44.957512,-74.379128,7.19,8.63,8.80,9.62,10.29,10.27,10.91,9.63,...,Z1,Z11,1,1,1a,Plaine du bas Outaouais et de l'archipel de Mo...,1216980.4,8.484385,1.396666,"POLYGON ((-74.42079 44.99568, -74.42079 44.999..."
4,44.957512,-74.295799,7.15,8.59,8.76,9.58,10.25,10.24,10.87,9.59,...,Z1,Z11,1,1,1a,Plaine du bas Outaouais et de l'archipel de Mo...,1216980.4,8.484385,1.396666,"POLYGON ((-74.33746 44.99204, -74.33746 44.999..."


## Create GeoJson Files

In [43]:
listTG = []
for tg in dftp[name].unique().tolist():
    df2 = dftp[dftp[name] == tg]
    print (tg)
    if tg != None:
        listTG.append(df2)
listTG

05151
07151
07251
06151
03571
06452
06271
07351
07152
08151
06471
04151
03451
08152
07352
03171
07451
04352
04251
01171
03153
08351
02371
03351
08251
04351
01272
08451
02571
11161
09751
11262
08462
08651
02651
08751
08551
02471
11263
08762
02751
02664
09351
08652
02666
08763
09471
08764
02665
08665
02663
08562
08664
08666
02662
08663
09551
02661
09352


[           lat        lon  hist_p50  t2050_rcp45_p10  t2080_rcp45_p10  \
 0    45.207500 -71.129257      2.68             4.14             4.31   
 1    45.207500 -70.879272      2.80             4.28             4.44   
 2    45.207500 -70.795937      2.08             3.54             3.71   
 3    45.290829 -71.212593      4.05             5.50             5.69   
 4    45.290829 -71.129257      3.78             5.24             5.42   
 ..         ...        ...       ...              ...              ...   
 147  45.790810 -70.545952      4.06             5.58             5.74   
 148  45.790810 -70.545952      4.06             5.58             5.74   
 149  45.790810 -70.462624      3.66             5.16             5.32   
 151  45.790810 -70.379288      3.18             4.68             4.83   
 204  45.874142 -71.295921      4.63             6.10             6.31   
 
      t2050_rcp45_p50  t2080_rcp45_p50  t2050_rcp45_p90  t2080_rcp45_p90  \
 0               5.04             

In [44]:
for tg in listTG:
    geometry = tg["geometry"]
    crs = {'init': "epsg:4326"}
    gdf = GeoDataFrame(tg, crs=crs, geometry=geometry)
    print (gdf[name].iloc[0])
    gdf.to_file(gdf[name].iloc[0]+"_tg_mean_annual.json", driver="GeoJSON")

05151
07151
07251
06151
03571
06452
06271
07351
07152
08151
06471
04151
03451
08152
07352
03171
07451
04352
04251
01171
03153
08351
02371
03351
08251
04351
01272
08451
02571
11161
09751
11262
08462
08651
02651
08751
08551
02471
11263
08762
02751
02664
09351
08652
02666
08763
09471
08764
02665
08665
02663
08562
08664
08666
02662
08663
09551
02661
09352


# Create geojson file for all regions

In [45]:
geometry = dftp["geometry"]
crs = {'init': "epsg:4326"}
gdf = GeoDataFrame(dftp, crs=crs, geometry=geometry)
gdf.to_file(short+"_tg_mean_annual.json", driver="GeoJSON")